&nbsp;

# 17 - Mémoire et fermeture markovienne de l'état réduit

---

&nbsp;

## 1. Introduction et objectif

Toujours dans l'optique de nous aiguiller sur la classe de modèles SciML "admissibles", nous pouvons chercher une dernière distinction.

&nbsp;

> #### L'état réduit est-il suffisant pour décrire la dynamique ?

C'est une question fondamentale sur laquelle nous n'avons pas encore tranchée (et sur laquelle on ne peut pas pour le moment).

Répondre à cette question revient à s'interroger sur ce qu'on appelle la fermeture dynamique en étude des systèmes. Étudier la fermeture consiste à chercher des dépendances ou des excitations du système que l'on souhaite approximer. L'hypothèse fondamentale de notre état est qu'il est parfaitement "fermée", soit :

$$
\dot{x}(t) = f(x(t))
$$

Si l'hypothèse d'un défaut de fermeture est admise, donc que le système est présumé dépendant/exciter par quelconques facteurs, alors notre état peut ressembler à :

$$
\text{(1)} \space \dot{x}(t) = f(x(t)) + g(t)
$$

ou :

$$
\text{(2)} \space \dot{x}(t) = f(x(t), x(t-\tau))
$$

ou encore :

$$
\text{(3)} \space \dot{x}(t) = f(x(t), u(t))
$$

De ces possiblités découlent alors une autre interrogation.

&nbsp;

> #### La problème de fermeture est-il de nature temporelle ou structurelle ?

Si le défaut est de nature temporelle alors on émet l'hypothèse que l'état est incomplet dans le temps. Et s'il est incomplet dans le temps, cela ne peut dire qu'une chose que le passé transporte de l'information dynamique, on parle dans ce cas là de *non-markovianité* en témoignage aux processus markoviens décrivant les systèmes ne dépendant pas des valeurs passées pour déterminer les futures. Cela correspond à notre cas $\text{(1)}$ et c'est l'hypothèse que nous allons testé dans ce notebook.

Si ce notebook indique un défaut de fermeture mais invalide l'hypothèse de *non-markovianité*, correspondant à un défaut de fermeture temporel, cela signifie probablement qu'un défaut de fermeture structurel est en cause, les équations $\text{(2)}$ et $\text{(3)}$ dans notre cas. Dans ce scénario nous étudierons cette nouvelle hypothèse dans un nouveau notebook dédié.

&nbsp;

---

&nbsp;

## 2. Autocorrélation des dérivées




